In [45]:
import torch
import numpy as np
import yfinance as yf
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import date
import pandas_market_calendars as mcal
import os
import csv

# Load your trained model
class StockPricePredictionModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.2):
        super(StockPricePredictionModel, self).__init__()
        self.gru = torch.nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = torch.nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.linear(out[:, -1, :])  # Use last GRU output
        return out

# --- Parameters ---
input_size = 5
hidden_size = 128
num_layers = 3
dropout = 0.2
sequence_length = 30  # Use last 30 days to predict next 5
prediction_steps = 5
features = ['Open', 'High', 'Low', 'Close', 'Volume']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_filename = 'output.pth'
capital_allocation = 10000  # The capital to allocate for buying/selling (e.g., $10,000)
initial_price_per_share = 278.71  # Assume the price per share when buying initially

# --- Load model ---
model = StockPricePredictionModel(input_size, hidden_size, num_layers, dropout).to(device)
model.load_state_dict(torch.load('./' + model_filename))  # Replace with actual saved file
model.eval()

# --- Download latest data ---
nyse = mcal.get_calendar('NYSE')
all_trading_days = nyse.valid_days(end_date='2025-03-28', start_date="2021-01-04")
last_trading_day = all_trading_days[-6].strftime('%Y-%m-%d')  # 5 days ago = index -6 (since it's inclusive)

start_date = '2021-01-03'  # end_date is the end of the 5 days
stock_data = yf.download('TSLA', start=start_date, end=last_trading_day)
stock_data = stock_data[features]

# --- Scale using only past data ---
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(stock_data)

# --- Prepare last 30 days for prediction ---
last_sequence = scaled_data[-sequence_length:]  # Shape: (30, 5)
predicted_open_prices = []

with torch.no_grad():
    for _ in range(prediction_steps):
        input_seq = torch.tensor(last_sequence, dtype=torch.float32).unsqueeze(0).to(device)  # (1, 30, 5)
        predicted_price = model(input_seq).cpu().numpy()[0, 0]
        predicted_open_prices.append(predicted_price)

        # Shift and update the last_sequence with predicted Open and repeated last values for other features
        last_known = last_sequence[-1]
        new_row = np.array([predicted_price, *last_known[1:]])  # keep High, Low, Close, Volume the same
        last_sequence = np.roll(last_sequence, shift=-1, axis=0)
        last_sequence[-1] = new_row

# --- Inverse transform just the Open values ---
predicted_array = np.array(predicted_open_prices).reshape(-1, 1)
padding = np.zeros((predicted_array.shape[0], input_size - 1))  # fill other columns with 0s
full_forecast_scaled = np.concatenate([predicted_array, padding], axis=1)
predicted_open_prices_scaled = scaler.inverse_transform(full_forecast_scaled)[:, 0]

# --- Print results for the next day only ---
# Generate next 5 NYSE trading days after the last_trading_day
forecast_dates = nyse.valid_days(start_date=pd.to_datetime(last_trading_day) + pd.Timedelta(days=1), end_date='2025-12-31')[:prediction_steps]

# Only print the predicted price for the next day
print(f"\nPredicted Open price for next day ({forecast_dates[0].strftime('%Y-%m-%d')}): ${predicted_open_prices_scaled[0]:.2f}")

# --- Trading Decision (updated logic with dynamic selling percentage) ---
def evaluate_trading_strategy(predicted_prices, capital_allocation, shares_owned):
    initial_price = predicted_prices[0]  # Use the first predicted price as initial price
    future_average_price = np.mean(predicted_prices[1:])  # Average price for the next days
    print(f"\n📊 Current Price: ${initial_price:.2f}, Average Future Price: ${future_average_price:.2f}")

    # Calculate the max shares you can buy with the available capital
    max_shares_to_buy = capital_allocation // initial_price  # How many shares can we afford to buy
    capital_to_invest = max_shares_to_buy * initial_price  # The capital required to buy these shares
    percentage_of_capital_to_invest = (capital_to_invest / capital_allocation) * 100  # Percentage of capital to invest

    # If price is expected to rise, buy shares
    if future_average_price > initial_price * 1.03:  # Buy threshold set to 3% increase
        shares_owned += max_shares_to_buy
        capital_allocation -= capital_to_invest
        return f"🚀 BUY: The market is likely to rise sharply. Consider investing {percentage_of_capital_to_invest:.2f}% of your capital!", capital_allocation, shares_owned

    # If price is expected to fall, sell shares (dynamic percentage based on price drop)
    elif future_average_price < initial_price * 0.97:  # Sell threshold set to 3% decrease
        # Calculate percentage of shares to sell based on how much the price has fallen
        price_drop_percentage = (initial_price - future_average_price) / initial_price * 100
        
        # Dynamic logic for the percentage of shares to sell
        sell_percentage = min(price_drop_percentage, 50)  # Cap at 50% max for example, can be adjusted
        shares_to_sell = shares_owned * (sell_percentage / 100)
        capital_from_sale = shares_to_sell * initial_price
        shares_owned -= shares_to_sell
        capital_allocation += capital_from_sale
        return f"📉 SELL: A decline in price is expected ({price_drop_percentage:.2f}% drop). Consider selling {sell_percentage:.2f}% of your shares.", capital_allocation, shares_owned

    # If no significant price movement expected, hold
    else:
        return f"🤔 HOLD: Price is expected to stay within a stable range, no major movement. You could consider holding {percentage_of_capital_to_invest:.2f}%.", capital_allocation, shares_owned

# Example usage
shares_owned = 100  # Assume 100 shares owned initially
strategy, updated_capital, updated_shares = evaluate_trading_strategy(predicted_open_prices_scaled, capital_allocation, shares_owned)
print(f"\n💡 Strategy Recommendation: {strategy}")

# --- Save the decision and capital change to CSV ---
def save_decision_to_csv(forecast_date, strategy, updated_capital):
    csv_filename = 'capital_changes.csv'
    header = ['date', 'strategy', 'updated_capital']

    # Check if file exists and append or create new
    file_exists = os.path.isfile(csv_filename)

    with open(csv_filename, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        if not file_exists:
            writer.writeheader()
        writer.writerow({'date': forecast_date, 'strategy': strategy, 'updated_capital': updated_capital})

# Save the decision for today
save_decision_to_csv(forecast_dates[0].strftime('%Y-%m-%d'), strategy, updated_capital)


C:\Users\erdmr\AppData\Local\Temp\ipykernel_29692\2825930013.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./' + model_filename))  #


Predicted Open price for next day (2025-03-24): $307.76

📊 Current Price: $307.76, Average Future Price: $280.86

💡 Strategy Recommendation: 📉 SELL: A decline in price is expected (8.74% drop). Consider selling 8.74% of your shares.
